Aby uruchomić poniższe notatniki, jeśli jeszcze tego nie zrobiłeś, musisz wdrożyć model, który używa `text-embedding-ada-002` jako modelu bazowego i ustawić nazwę wdrożenia w pliku .env jako `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Następnie załadujemy Indeks Osadzeń do ramki danych Pandas. Indeks Osadzeń jest zapisany w pliku JSON o nazwie `embedding_index_3m.json`. Indeks Osadzeń zawiera osadzenia dla każdego z transkryptów YouTube aż do końca października 2023 roku.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Następnie stworzymy funkcję o nazwie `get_videos`, która będzie wyszukiwać zapytanie w Indeksie Osadzeń. Funkcja zwróci 5 filmów najbardziej podobnych do zapytania. Działanie funkcji wygląda następująco:

1. Najpierw tworzona jest kopia Indeksu Osadzeń.
2. Następnie, dla zapytania obliczane jest osadzenie przy użyciu OpenAI Embedding API.
3. Potem w Indeksie Osadzeń tworzona jest nowa kolumna o nazwie `similarity`. Kolumna `similarity` zawiera wartość kosinusowej miary podobieństwa pomiędzy osadzeniem zapytania a osadzeniem każdego segmentu wideo.
4. Następnie Indeks Osadzeń jest filtrowany według kolumny `similarity`. Indeks zostaje ograniczony tylko do tych filmów, które mają kosinusową miarę podobieństwa większą lub równą 0,75.
5. Na końcu Indeks Osadzeń jest sortowany według kolumny `similarity` i zwracane jest 5 najlepszych filmów.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Ta funkcja jest bardzo prosta, po prostu wyświetla wyniki zapytania wyszukiwania.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Najpierw indeks osadzeń jest ładowany do DataFrame Pandas.
2. Następnie użytkownik zostaje poproszony o wpisanie zapytania.
3. Potem wywoływana jest funkcja `get_videos`, aby przeszukać indeks osadzeń pod kątem zapytania.
4. Na końcu wywoływana jest funkcja `display_results`, która wyświetla wyniki użytkownikowi.
5. Użytkownik jest ponownie proszony o wpisanie kolejnego zapytania. Proces ten powtarza się, dopóki użytkownik nie wpisze `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.pl.png)

Zostaniesz poproszony o wpisanie zapytania. Wpisz zapytanie i naciśnij enter. Aplikacja zwróci listę filmów powiązanych z zapytaniem. Otrzymasz także link do miejsca w filmie, gdzie znajduje się odpowiedź na Twoje pytanie.

Oto przykładowe zapytania, które możesz wypróbować:

- Czym jest Azure Machine Learning?
- Jak działają konwolucyjne sieci neuronowe?
- Co to jest sieć neuronowa?
- Czy mogę używać Jupyter Notebooks z Azure Machine Learning?
- Co to jest ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony przy użyciu usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby zapewnić poprawność tłumaczenia, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być traktowany jako źródło nadrzędne. W przypadku informacji o kluczowym znaczeniu zalecamy skorzystanie z profesjonalnych usług tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z korzystania z tego tłumaczenia.
